In [ ]:
import warnings
import pandas as pd
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
!pip install gdown
import gdown

In [ ]:
file_id = '1MlvBT3CwGKJ_JYIPFeGVhwfLmEQu513c'
url = f'https://drive.google.com/uc?id={file_id}'

output = 'dataset.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1MlvBT3CwGKJ_JYIPFeGVhwfLmEQu513c
To: /content/dataset.csv
100%|██████████| 3.90M/3.90M [00:00<00:00, 157MB/s]


'dataset.csv'

In [ ]:
df = pd.read_csv('dataset.csv', compression='gzip')
df.shape
df.head()
df.dtypes
df.isna().mean().sort_values(ascending=False)

,0
Sunshine,0.480098
Evaporation,0.431665
Cloud3pm,0.408071
Cloud9am,0.384216
Pressure9am,0.103568
Pressure3pm,0.103314
WindDir9am,0.072639
WindGustDir,0.070989
WindGustSpeed,0.070555
Humidity3pm,0.030984


3.1. Видаліть із набору ознаки з великою кількістю пропущених значень.

In [ ]:
df = df.dropna(subset=['RainTomorrow'])
df = df[df.columns[df.isna().mean().lt(0.35)]]

3.2. Створіть підмножини набору даних із числовими та категоріальними ознаками.

3.3. Змініть тип колонки Date на тип datetimeі створіть додаткові колонки Year та Month.

3.4. Переміcтить створену нову колонку Year з підмножини набору із категоріальними ознаками до підмножини із числовими ознаками.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
data_num = df.select_dtypes(include=np.number)
data_cat = df.select_dtypes(include='object')

3.5. Розбийте підмножини на тренувальну і тестову вибірки за такою логікою: до тестової вибірки віднесіть всі об'єкти із набору даних із останнім (максимальним) роком спостережень, а для навчання моделі залиште всі інші об'єкти.

In [ ]:
max_year = df['Year'].max()

train_data = df[df['Year'] < max_year]
test_data = df[df['Year'] == max_year]

X_train_num = train_data.select_dtypes(include=np.number).drop(columns=['Year'])
X_test_num = test_data.select_dtypes(include=np.number).drop(columns=['Year'])

X_train_cat = train_data.select_dtypes(include='object').drop(columns=['RainTomorrow'])
X_test_cat = test_data.select_dtypes(include='object').drop(columns=['RainTomorrow'])

y_train = train_data['RainTomorrow']
y_test = test_data['RainTomorrow']

print("X_train_num shape:", X_train_num.shape)
print("X_test_num shape:", X_test_num.shape)
print("X_train_cat shape:", X_train_cat.shape)
print("X_test_cat shape:", X_test_cat.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train_num shape: (133727, 13)
X_test_num shape: (8466, 13)
X_train_cat shape: (133727, 5)
X_test_cat shape: (8466, 5)
y_train shape: (133727,)
y_test shape: (8466,)



4. Відновіть пропущені дані за допомогою об'єкта SimpleImputer з пакету sklearn.



In [ ]:
num_imputer = SimpleImputer(strategy='mean')
X_train_num = pd.DataFrame(num_imputer.fit_transform(X_train_num), columns=X_train_num.columns)
X_test_num = pd.DataFrame(num_imputer.transform(X_test_num), columns=X_test_num.columns)

cat_imputer = SimpleImputer(strategy='most_frequent')
X_train_cat = pd.DataFrame(cat_imputer.fit_transform(X_train_cat), columns=X_train_cat.columns)
X_test_cat = pd.DataFrame(cat_imputer.transform(X_test_cat), columns=X_test_cat.columns)

print("Missing values in training numerical features:", X_train_num.isna().sum().sum())
print("Missing values in testing numerical features:", X_test_num.isna().sum().sum())
print("Missing values in training categorical features:", X_train_cat.isna().sum().sum())
print("Missing values in testing categorical features:", X_test_cat.isna().sum().sum())

Missing values in training numerical features: 0
Missing values in testing numerical features: 0
Missing values in training categorical features: 0
Missing values in testing categorical features: 0


5. Нормалізуйте числові ознаки за допомогою об'єкта StandardScaler з пакету sklearn.



In [ ]:
scaler = StandardScaler()

X_train_num_scaled = pd.DataFrame(scaler.fit_transform(X_train_num), columns=X_train_num.columns)
X_test_num_scaled = pd.DataFrame(scaler.transform(X_test_num), columns=X_test_num.columns)

print(X_train_num_scaled.head())
print(X_test_num_scaled.head())

    MinTemp   MaxTemp  Rainfall  WindGustSpeed  WindSpeed9am  WindSpeed3pm  \
0  0.205482 -0.030108 -0.207381       0.292771      0.667526      0.602373   
1 -0.736944  0.279808 -0.278796       0.292771     -1.134272      0.373916   
2  0.126946  0.364331 -0.278796       0.444575      0.554913      0.830830   
3 -0.454216  0.688335 -0.278796      -1.225267     -0.345986     -1.111055   
4  0.849473  1.294081 -0.159771       0.065065     -0.796435      0.145459   

   Humidity9am  Humidity3pm  Pressure9am  Pressure3pm   Temp9am   Temp3pm  \
0     0.121932    -1.433143    -1.470784    -1.216395 -0.001583  0.031333   
1    -1.303491    -1.287215    -1.040659    -1.111524  0.044840  0.395202   
2    -1.620251    -1.044002    -1.485616    -0.976689  0.632863  0.235100   
3    -1.250697    -1.724999    -0.002427    -0.362443  0.184108  0.715406   
4     0.702659    -0.898074    -1.010995    -1.381193  0.137685  1.181158   

      Month  
0  1.580628  
1  1.580628  
2  1.580628  
3  1.580628 

6. Виконайте кодування категоріальних ознак за допомогою об’єкта OneHotEncoder з пакету sklearn.


In [ ]:
encoder = (OneHotEncoder(drop='if_binary',
                         sparse_output=False)
           .set_output(transform='pandas'))

X_train_cat_encoded = pd.DataFrame(encoder.fit_transform(X_train_cat),
                                   columns=encoder.get_feature_names_out(X_train_cat.columns))
X_test_cat_encoded = pd.DataFrame(encoder.transform(X_test_cat),
                                  columns=encoder.get_feature_names_out(X_test_cat.columns))

print(X_train_cat_encoded.head())
print(X_test_cat_encoded.head())

   Location_Adelaide  Location_Albany  Location_Albury  Location_AliceSprings  \
0                0.0              0.0              1.0                    0.0   
1                0.0              0.0              1.0                    0.0   
2                0.0              0.0              1.0                    0.0   
3                0.0              0.0              1.0                    0.0   
4                0.0              0.0              1.0                    0.0   

   Location_BadgerysCreek  Location_Ballarat  Location_Bendigo  \
0                     0.0                0.0               0.0   
1                     0.0                0.0               0.0   
2                     0.0                0.0               0.0   
3                     0.0                0.0               0.0   
4                     0.0                0.0               0.0   

   Location_Brisbane  Location_Cairns  Location_Canberra  ...  WindDir3pm_NW  \
0                0.0              0.

7. Об'єднайте підмножини з числовими і категоріальними ознаками (після кодування) для побудови моделі за допомогою об’єкта LogisticRegression з пакету sklearn.

In [ ]:
X_train_combined = pd.concat([X_train_num_scaled, X_train_cat_encoded], axis=1)
X_test_combined = pd.concat([X_test_num_scaled, X_test_cat_encoded], axis=1)

print("X_train_combined shape:", X_train_combined.shape)
print("X_test_combined shape:", X_test_combined.shape)

X_train_combined shape: (133727, 111)
X_test_combined shape: (8466, 111)


8. Розрахуйте метрики нової моделі за допомогою методу classification_report() з пакета sklearn, порівняйте із метриками моделі, отриманими в розділі «Практика застосування логістичної регресії. Навчання й оцінка моделі. Оцінювання точності моделі» теми «Логістична регресія. Оцінка якості класифікації», зробіть висновки.


In [ ]:
solvers = ['liblinear', 'lbfgs', 'saga']

solver_reports = {}

for solver in solvers:
    print(f"\nMetrics for solver: {solver}")

    clf = LogisticRegression(solver=solver, max_iter=1000, random_state=42)
    clf.fit(X_train_combined, y_train)

    y_pred = clf.predict(X_test_combined)

    report = classification_report(y_test, y_pred, output_dict=True)
    solver_reports[solver] = report

    print(classification_report(y_test, y_pred))


Metrics for solver: liblinear
              precision    recall  f1-score   support

          No       0.87      0.96      0.91      6703
         Yes       0.72      0.44      0.55      1763

    accuracy                           0.85      8466
   macro avg       0.79      0.70      0.73      8466
weighted avg       0.84      0.85      0.83      8466


Metrics for solver: lbfgs
              precision    recall  f1-score   support

          No       0.87      0.96      0.91      6703
         Yes       0.72      0.44      0.55      1763

    accuracy                           0.85      8466
   macro avg       0.79      0.70      0.73      8466
weighted avg       0.84      0.85      0.83      8466


Metrics for solver: saga
              precision    recall  f1-score   support

          No       0.87      0.96      0.91      6703
         Yes       0.72      0.44      0.55      1763

    accuracy                           0.85      8466
   macro avg       0.79      0.70      0.73 

### Звіт про класифікацію

| Метрика          | Клас "Ні" | Клас "Так" | Точність | Середнє по класах (macro avg) | Зважене середнє (weighted avg) | Кількість (support) |
|------------------|------------|------------|----------|-------------------------------|--------------------------------|---------------------|
| **liblinear**     |            |            |          |                               |                                |                     |
| Точність (precision)  | 0.87       | 0.72       | 0.85     | 0.79                          | 0.84                           | 8466                |
| Повнота (recall)      | 0.96       | 0.44       |          | 0.70                          | 0.85                           |                     |
| F1-оцінка (f1-score)  | 0.91       | 0.55       |          | 0.73                          | 0.83                           |                     |
| **lbfgs**         |            |            |          |                               |                                |                     |
| Точність (precision)  | 0.87       | 0.72       | 0.85     | 0.79                          | 0.84                           | 8466                |
| Повнота (recall)      | 0.96       | 0.44       |          | 0.70                          | 0.85                           |                     |
| F1-оцінка (f1-score)  | 0.91       | 0.55       |          | 0.73                          | 0.83                           |                     |
| **saga**          |            |            |          |                               |                                |                     |
| Точність (precision)  | 0.87       | 0.72       | 0.85     | 0.79                          | 0.84                           | 8466                |
| Повнота (recall)      | 0.96       | 0.44       |          | 0.70                          | 0.85                           |                     |
| F1-оцінка (f1-score)  | 0.91       | 0.55       |          | 0.73                          | 0.83                           |                     |

### Основні спостереження:

- **Точність (precision)** для класу "Ні" висока для всіх солверів (0.87).
- **Повнота (recall)** для класу "Так" є відносно низькою (0.44), що означає, що модель може пропускати деякі позитивні випадки.
- **Точність (accuracy)** стабільна для всіх солверів (0.85), що трохи краще, ніж очікувані 0.80.
- **Середнє по класах (macro avg)** та **зважене середнє (weighted avg)** мають схожі значення, що свідчить про збалансовану роботу моделі для обох класів.

### Висновки:

- Всі солвери (`liblinear`, `lbfgs`, та `saga`) показують подібні результати, без значних відмінностей у точності, повноті чи F1-оцінці.
- Хоча загальна точність моделі є достатньо високою, низька повнота для класу "Так" свідчить про труднощі з ідентифікацією позитивних випадків.
- Можна спробувати налаштувати модель далі або дослідити інші моделі, щоб покращити показники для класу "Так", залежно від того, наскільки важливим є коректне ідентифікування цього класу.

**Щоб покращити показники для класу "Так", можна спробувати налаштувати гіперпараметри логістичної регресії:**


 * Балансування класів: Оскільки клас "Так" менш поширений, можна використовувати аргумент class_weight='balanced', щоб модель приділяла більше уваги цьому класу.

* Регуляризація: Спробуй налаштувати параметр регуляризації C. За замовчуванням значення C=1.0. Зменшення C може допомогти знизити дисперсію моделі.

In [ ]:
# Налаштування з class_weight='balanced' та іншим значенням параметра C
clf = LogisticRegression(solver='liblinear', class_weight='balanced', C=0.5, max_iter=1000, random_state=42)
clf.fit(X_train_combined, y_train)
y_pred = clf.predict(X_test_combined)

# Оцінка метрик
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          No       0.92      0.82      0.87      6703
         Yes       0.52      0.71      0.60      1763

    accuracy                           0.80      8466
   macro avg       0.72      0.77      0.73      8466
weighted avg       0.83      0.80      0.81      8466



### Висновок порівняння першої логістичної регресії та після регулювання гіперпараметрів

Після регулювання гіперпараметрів логістичної регресії ми спостерігаємо значне покращення метрик для класу "Так", хоча це супроводжується певними компромісами в інших аспектах.

#### Основні зміни:

1. **Повнота (recall) класу "Так"**:
   - **До налаштування**: Повнота для класу "Так" становила лише **0.44**, що свідчить про те, що модель пропускала значну кількість позитивних випадків.
   - **Після налаштування**: Повнота покращилася до **0.71**, що означає, що модель значно краще ідентифікує позитивні випадки. Це велике покращення і досягає головної мети налаштування.

2. **Точність (precision) класу "Так"**:
   - **До налаштування**: Точність була на рівні **0.72**, що означало менше хибнопозитивних результатів (помилкових ідентифікацій класу "Так").
   - **Після налаштування**: Точність впала до **0.52**, що вказує на збільшення кількості хибнопозитивних випадків, тобто модель частіше помиляється, коли прогнозує клас "Так".

3. **F1-оцінка для класу "Так"**:
   - **До налаштування**: F1-оцінка для класу "Так" становила **0.55**, що є збалансованою оцінкою між точністю та повнотою.
   - **Після налаштування**: F1-оцінка покращилася до **0.60**, що свідчить про покращення загальної продуктивності для класу "Так" з більшою орієнтацією на повноту.

4. **Точність моделі (accuracy)**:
   - **До налаштування**: Загальна точність моделі була **0.85**.
   - **Після налаштування**: Точність знизилася до **0.80**, що є очікуваним, оскільки модель тепер намагається знайти більше позитивних випадків, що може знижувати загальну точність через більшу кількість хибнопозитивних результатів.

#### Висновок:
Налаштування гіперпараметрів дозволило значно покращити **повноту** для класу "Так", що означає, що модель краще ідентифікує позитивні випадки. Проте це досягнення було досягнуто за рахунок зниження **точності**, що призвело до збільшення кількості хибнопозитивних результатів. Загальна **точність моделі** також знизилася, але це допустимо, якщо головним пріоритетом є знаходження більшої кількості позитивних випадків.

Якщо важливо, щоб модель виявляла максимальну кількість позитивних випадків (навіть якщо при цьому буде більше хибнопозитивних), то налаштована логістична регресія є кращим варіантом. Однак, якщо важливіша **точність** і уникнення хибнопозитивних прогнозів, то оригінальна модель або інші підходи можуть бути кращими.